In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix 
from time import perf_counter
from time import process_time
import timeit

In [ ]:
# loading the dataset to a pandas DataFrame 
#df = pd.read_csv('/kaggle/input/credit-card-undersample/creditcard.csv')
df = pd.read_csv('/kaggle/input/creditcard2023/creditcard_2023.csv')
df

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", 
      df.isnull().sum()) 


In [ ]:
target = 'Class'
X = df.loc[:, df.columns!=target]
Y = df.loc[:, df.columns==target]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=0.2, 
                                                    random_state=42)


ax = sns.countplot(x=target, data=df)
print(df[target].value_counts())
Y_train[target].value_counts()

In [ ]:
'''sm = SMOTE(random_state=12, sampling_strategy=1.0)
x_train_res, y_train_res = sm.fit_resample(X_train, Y_train)
unique, count = np.unique(y_train_res, return_counts=True)
y_train_smote_value_count = { k:v for (k,v) in zip(unique, count)}
y_train_smote_value_count
X=x_train_res
Y=y_train_res'''

In [ ]:
ax = sns.countplot(x=target, data=Y_train)
print(df[target].value_counts())
Y_train[target].value_counts()

# SMOTETomek

In [ ]:
'''from imblearn.combine import SMOTETomek

s_tomek = SMOTETomek(random_state = 42)
x_train_res, y_train_res = s_tomek.fit_resample(X_train,Y_train)
unique, count = np.unique(y_train_res, return_counts=True)
y_train_smote_value_count = { k:v for (k,v) in zip(unique, count)}
y_train_smote_value_count'''

In [ ]:
X_train.shape
print(Y['Class'].value_counts())

In [ ]:
X.shape

In [ ]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print("Precision = " ,precision_score(y_actual, y_predicted))
    print("Recall = " ,recall_score(y_actual, y_predicted))
    print("F1 Score = " ,f1_score(y_actual, y_predicted))
    pass


def generate_auc_roc_curve(clf, X_test):
    y_pred_proba = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(Y_test,  y_pred_proba)
    auc = roc_auc_score(Y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
    plt.legend(loc=4)
    plt.show()
    pass

In [ ]:
X.corr()

import seaborn as sns
#Using Pearson Correlation
plt.figure(figsize=(20,30))
cor = X.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.autumn)
plt.show()
def correlation(dataset, threshold):#dataset is X_train , threshold is a value which is may be 80% ,90% ,70%.. 
    col_corr = set()  # Set of all the names of correlated columns
    # using set() so no duplicate column contain here ...
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X, 0.95)

In [ ]:
len(set(corr_features))

In [ ]:
corr_features

In [ ]:
for col in corr_features:
    for col1 in X.columns:
        if col1==col:
            #print(col)
            del X_train[col]
            del X_test[col]
print(X_train)
print(X_test)

In [ ]:
#X_train=X

In [ ]:
X_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=4, 
                             random_state=2018,
                             criterion='gini',
                             n_estimators=100,
                             verbose=False)
rf.fit(X_train, Y_train)

In [ ]:
rf.feature_importances_

In [ ]:
#creating a table for checking the importance of different variables  
feat_imp=pd.DataFrame({"Features":X_train.columns,"Importance":rf.feature_importances_}).sort_values(by="Importance",ascending=False)


In [ ]:
feat_imp

In [ ]:
## visualizing the importance of variables
plt.figure(figsize=(12,6))
sns.barplot(data=feat_imp,x="Features",y="Importance")
plt.show()

In [ ]:
#filtering out the variables having more than 1% impact on the model
feat_imp[feat_imp["Importance"]>0.015]["Features"].unique()

In [ ]:
## selecting the important variabels
X_train=X_train[['id', 'V14', 'V12', 'V4', 'V11', 'V10', 'V17', 'V16', 'V18', 'V3']]

X_test=X_test[['id', 'V14', 'V12', 'V4', 'V11', 'V10', 'V17', 'V16', 'V18', 'V3']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max = MinMaxScaler()
X_train = pd.DataFrame(min_max.fit_transform(X_train), columns=X_train.columns)
X_train

In [ ]:
X_test = pd.DataFrame(
    min_max.transform(X_test),
    columns = X_test.columns
)

In [ ]:
#X=X_train

# Correlation

In [ ]:
'''X.corr()

import seaborn as sns
#Using Pearson Correlation
plt.figure(figsize=(12,20))
cor = X.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.autumn)
plt.show()
def correlation(dataset, threshold):#dataset is X_train , threshold is a value which is may be 80% ,90% ,70%.. 
    col_corr = set()  # Set of all the names of correlated columns
    # using set() so no duplicate column contain here ...
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X, 0.85)'''

In [ ]:
#len(set(corr_features))

In [ ]:
#corr_features

In [ ]:
'''for col in corr_features:
    for col1 in X.columns:
        if col1==col:
            #print(col)
            del X[col]
            del X_test[col]
print(X)
print(X_test)'''

# Hyperparameter for Decision Tree

In [ ]:
dt_hp = tree.DecisionTreeClassifier(random_state=43)

params = {'max_depth':[2,10,15],
          'min_samples_leaf':[1,3,5],
          'min_samples_split':[1,2,5,8],
          'criterion':['gini','entropy']}
GS = GridSearchCV(estimator=dt_hp,param_grid=params,cv=5,n_jobs=-1, verbose=True, scoring='accuracy')

In [ ]:
#GS.fit(X_train, Y_train)

In [ ]:
#print('Best Parameters:',GS.best_params_,end='\n\n')
#print('Best Score:',GS.best_score_)

In [ ]:
from sklearn import tree
classify_ = tree.DecisionTreeClassifier(criterion= 'entropy', max_depth= 10, min_samples_leaf= 6, min_samples_split= 3)
start_time = timeit.default_timer()
classify_ = classify_.fit(X_train,Y_train)

end_time = timeit.default_timer()
elapsed_time = end_time - start_time

print("Elapsed time : ",elapsed_time)

#Accuracy on training data
x_train_prediction = classify_.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction , Y_train)
print('Accuracy on Training data DT: , ', training_data_accuracy)

# accuracy on test data 
x_test_prediction = classify_.predict(X_test)
test_data_accuracy=accuracy_score(x_test_prediction,Y_test)
print('Accuracy score on Test data DT:' ,test_data_accuracy)

generate_model_report(Y_test, x_test_prediction)
cf_matrix=confusion_matrix(x_test_prediction,Y_test)
print(cf_matrix)
sns.heatmap(cf_matrix, annot=True, cmap='Blues',fmt='g')

# Hyperparameter for XGBoost

In [ ]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from xgboost import XGBClassifier
model = XGBClassifier()
params ={'max_depth':[1,10,20,50],
        'gamma':[0],
        'reg_alpha' :[ 1],
        'reg_lambda' :[1],
        'colsample_bytree' :  [0,1],
        'min_child_weight' : [1],
        'n_estimators': [1,100,200],
        'seed': [0]}
GS = GridSearchCV(estimator=model,param_grid=params,cv=3,n_jobs=-1, verbose=True, scoring='f1')

In [ ]:
GS.fit(X_train, Y_train)

In [ ]:
print('Best Parameters:',GS.best_params_,end='\n\n')
print('Best Score:',GS.best_score_)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(colsample_bytree= 1 , gamma = 0 , max_depth= 10, min_child_weight= 1, n_estimators=100, reg_alpha= 1, reg_lambda= 1, seed= 0)
start_time = timeit.default_timer()
model.fit(X_train, Y_train)
end_time = timeit.default_timer()
elapsed_time = end_time - start_time

print("Elapsed time : ",elapsed_time)

#Accuracy on training data
x_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction , Y_train)
print('Accuracy on Training data : , ', training_data_accuracy)

# accuracy on test data 
x_test_prediction = model.predict(X_test)
test_data_accuracy=accuracy_score(x_test_prediction,Y_test)
print('Accuracy score on Test data XGBost:' ,test_data_accuracy)
generate_model_report(Y_test, x_test_prediction)

confusion_matrix(x_test_prediction,Y_test)
cf_matrix=confusion_matrix(x_test_prediction,Y_test)
print(cf_matrix)
sns.heatmap(cf_matrix, annot=True, cmap='Blues',fmt='g')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=4,random_state=2018,criterion='entropy',n_estimators=100,verbose=False)
start_time = timeit.default_timer()
rf.fit(X_train, Y_train)

end_time = timeit.default_timer()
elapsed_time = end_time - start_time

print("Elapsed time : ",elapsed_time)

x_train_prediction = rf.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction , Y_train)
print('Accuracy on Training data : , ', training_data_accuracy)

# accuracy on test data 
x_test_prediction = rf.predict(X_test)
test_data_accuracy=accuracy_score(x_test_prediction,Y_test)
print('Accuracy score on Test data RF :' ,test_data_accuracy)

generate_model_report(Y_test, x_test_prediction)
#print(confusion_matrix(x_test_prediction,Y_test))
confusion_matrix(x_test_prediction,Y_test)
cf_matrix=confusion_matrix(x_test_prediction,Y_test)
print(cf_matrix)
sns.heatmap(cf_matrix, annot=True, cmap='Blues',fmt='g')

# Hyperparameter for ANN

In [ ]:
from sklearn.neural_network import MLPClassifier
#{'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (10, 30, 30), 'learning_rate': 'invscaling', 'max_iter': 10000, 'solver': 'adam'}

MLPC = MLPClassifier()
params = {
    'hidden_layer_sizes': [(10,30,10),(20,20,10),(20,40),(30,20,10)]
    #'activation': ['tanh'],
    #'solver': [ 'adam'],
    #'alpha': [0.0001],
    #'learning_rate': ['adaptive'],
    #'max_iter':[10000,30000]
}

In [ ]:
GS = GridSearchCV(estimator=MLPC,param_grid=params,cv=3,n_jobs=-1, verbose=True, scoring='recall')
GS.fit(X_train, Y_train)
print('Best Parameters:',GS.best_params_,end='\n\n')
print('Best Score:',GS.best_score_)

In [ ]:
from sklearn.neural_network import MLPClassifier

MLPC = MLPClassifier(activation= 'tanh', alpha= 0.0001, hidden_layer_sizes=(12, 40, 30), learning_rate= 'adaptive', max_iter= 4000, solver= 'adam')
start_time = timeit.default_timer()
MLPC.fit(X_train,Y_train)
end_time = timeit.default_timer()
elapsed_time = end_time - start_time

print("Elapsed time : ",elapsed_time)

#Accuracy on training data
x_train_prediction = MLPC.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction , Y_train)
print('Accuracy on Training data : , ', training_data_accuracy)

# accuracy on test data 
x_test_prediction = MLPC.predict(X_test)
test_data_accuracy=accuracy_score(x_test_prediction,Y_test)
print('Accuracy score on Test data ANN :' ,test_data_accuracy)

generate_model_report(Y_test, x_test_prediction)
#print(confusion_matrix(x_test_prediction,Y_test))
confusion_matrix(x_test_prediction,Y_test)
cf_matrix=confusion_matrix(x_test_prediction,Y_test)
print(cf_matrix)
sns.heatmap(cf_matrix, annot=True, cmap='Blues',fmt='g')

# Hyperparameter for KNN

In [ ]:
n_neighbors=5, *, weights="uniform", algorithm="auto", leaf_size=30, p=2, metric="minkowski"

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 
model = KNeighborsClassifier()
params = { 'n_neighbors':[5,10,15], 'weights':['uniform', 'distance'], 'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'], 'leaf_size':[20,30,50],'metric':'['minkowski']
                }
GS = GridSearchCV(estimator=model,param_grid=params,cv=3,n_jobs=-1, verbose=True, scoring='accuracy')
GS.fit(X_train, Y_train)
print('Best Parameters:',GS.best_params_,end='\n\n')
print('Best Score:',GS.best_score_)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors= 9,metric='manhattan',weights='uniform')
start_time =timeit.default_timer()
knn.fit(X_train,Y_train)

end_time = timeit.default_timer()
elapsed_time = end_time - start_time

print("Elapsed time : ",elapsed_time)
#Accuracy on training data
x_train_prediction = knn.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction , Y_train)
print('Accuracy on Training data : , ', training_data_accuracy)
# accuracy on test data 
x_test_prediction  = knn.predict(X_test)
test_data_accuracy = accuracy_score(x_test_prediction,Y_test)
print('Accuracy score on Test data KNN :' ,test_data_accuracy)
Y_Test_Pred = knn.predict(X_test)
generate_model_report(Y_test, Y_Test_Pred)
confusion_matrix(x_test_prediction,Y_test)
cf_matrix=confusion_matrix(x_test_prediction,Y_test)
print(cf_matrix)
sns.heatmap(cf_matrix, annot=True, cmap='Blues',fmt='g')

In [ ]:
logModel = LogisticRegression()
start_time = perf_counter()
logModel.fit(X_train, Y_train)
end_time = perf_counter()
elapsed_time = end_time - start_time

print("Elapsed time : ",elapsed_time)

#Accuracy on training data
x_train_prediction = logModel.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction , Y_train)
print('Accuracy on Training data : , ', training_data_accuracy)

# accuracy on test data 
x_test_prediction = logModel.predict(X_test)
test_data_accuracy=accuracy_score(x_test_prediction,Y_test)
print('Accuracy score on Test data :' ,test_data_accuracy)
generate_model_report(Y_test, x_test_prediction)
print(confusion_matrix(x_test_prediction,Y_test))

# Catboost Hyperparameter

In [ ]:
from catboost import CatBoostClassifier
cb_clf = CatBoostClassifier()

params = {
    'learning_rate': [0.03, 0.06],
    'depth':[3, 9],
   # 'l2_leaf_reg': [2, 3, 4],
   # 'boosting_type': ['Ordered', 'Plain']
}

GS = GridSearchCV(estimator=cb_clf,param_grid=params,cv=3,n_jobs=-1, verbose=True, scoring='accuracy')
GS.fit(X_train, Y_train)
print('Best Parameters:',GS.best_params_,end='\n\n')
print('Best Score:',GS.best_score_)

In [ ]:
'''learning_rate': [0.03, 0.06],
    'depth':[3, 6, 9],
    'l2_leaf_reg': [2, 3, 4],
    'boosting_type': ['Ordered', 'Plain']'''

from catboost import CatBoostClassifier

cb_clf = CatBoostClassifier(learning_rate=0.03,depth=9)
cb_clf.fit(X_train, Y_train)
#y_train_pred = cb_clf.predict(X_train)
#y_test_pred = cb_clf.predict(X_test)

#Accuracy on training data
x_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction , Y_train)
print('Accuracy on Training data : , ', training_data_accuracy)

# accuracy on test data 
x_test_prediction = model.predict(X_test)
test_data_accuracy=accuracy_score(x_test_prediction,Y_test)
print('Accuracy score on Test data :' ,test_data_accuracy)
generate_model_report(Y_test, x_test_prediction)
confusion_matrix(x_test_prediction,Y_test)
cf_matrix=confusion_matrix(x_test_prediction,Y_test)
print(cf_matrix)
sns.heatmap(cf_matrix, annot=True, cmap='Blues')

In [ ]:
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB(binarize=0.0)
start_time = timeit.default_timer()
model.fit(X_train,Y_train)

end_time = timeit.default_timer()
elapsed_time = end_time - start_time

print("Elapsed time : ",elapsed_time)

#Accuracy on training data
x_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction , Y_train)
print('Accuracy on Training data : , ', training_data_accuracy)
# accuracy on test data 
x_test_prediction = model.predict(X_test)
test_data_accuracy=accuracy_score(x_test_prediction,Y_test)
print('Accuracy score on Test data :' ,test_data_accuracy)
generate_model_report(Y_test, x_test_prediction)
print(confusion_matrix(x_test_prediction,Y_test))

# ROC-AUC curve

In [ ]:
    y_pred_proba_dt = classify_.predict_proba(X_test)[:, 1]
    fpr_dt, tpr_dt, thresholds = roc_curve(Y_test,  y_pred_proba_dt)
    auc_dt = roc_auc_score(Y_test, y_pred_proba_dt)
    
    y_pred_proba_xb = model.predict_proba(X_test)[:, 1]
    fpr_xb, tpr_xb, thresholds = roc_curve(Y_test,  y_pred_proba_xb)
    auc_xb = roc_auc_score(Y_test, y_pred_proba_xb)
    
    y_pred_proba_rf = rf.predict_proba(X_test)[:, 1]
    fpr_rf, tpr_rf, thresholds = roc_curve(Y_test,  y_pred_proba_rf)
    auc_rf = roc_auc_score(Y_test, y_pred_proba_rf)
    
    y_pred_proba_ann = MLPC.predict_proba(X_test)[:, 1]
    fpr_ann, tpr_ann, thresholds = roc_curve(Y_test,  y_pred_proba_ann)
    auc_ann = roc_auc_score(Y_test, y_pred_proba_ann)
    
    y_pred_proba_knn = knn.predict_proba(X_test)[:, 1]
    fpr_knn, tpr_knn, thresholds = roc_curve(Y_test,  y_pred_proba_knn)
    auc_knn = roc_auc_score(Y_test, y_pred_proba_knn)
    
   # y_pred_proba_cb = cb_clf.predict_proba(X_test)[:, 1]
    #fpr_cb, tpr_cb, thresholds = roc_curve(Y_test,  y_pred_proba_cb)
    #auc_cb = roc_auc_score(Y_test, y_pred_proba_cb)
    
    plt.figure(figsize=(5, 5), dpi=100)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    
    plt.plot(fpr_dt,tpr_dt,linestyle='solid',label="DTC ="+str(auc_dt))
    plt.plot(fpr_xb,tpr_xb,linestyle='dotted',label="XGBC ="+str(auc_xb))
    plt.plot(fpr_rf,tpr_rf,linestyle='dashed',label="RFC ="+str(auc_rf))
    plt.plot(fpr_ann,tpr_ann,linestyle='-.',label="ANN ="+str(auc_ann))
    plt.plot(fpr_knn,tpr_knn,linestyle='dashdot',label="KNN ="+str(auc_knn))
    #plt.plot(fpr_cb,tpr_cb,linestyle=':',label="CatBoost ="+str(auc_cb))
    plt.legend(loc=4)
    plt.show()

# Precision Recall Curve

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn import metrics
#print('Logistic PR-AUC: %.3f'% auc_score)
y_test_prob_rf=rf.predict_proba(X_test)[:,1]
precision_rf, recall_rf,_ =precision_recall_curve(Y_test, y_test_prob_rf)
auc_score_rf=metrics.auc(recall_rf, precision_rf)
y_test_prob_xb=model.predict_proba(X_test)[:,1]
precision_xb, recall_xb,_ =precision_recall_curve(Y_test, y_test_prob_xb)
auc_score_xb=metrics.auc(recall_xb, precision_xb)
y_test_prob_dt=classify_.predict_proba(X_test)[:,1]
precision_dt, recall_dt,_ =precision_recall_curve(Y_test, y_test_prob_dt)
auc_score_dt=metrics.auc(recall_dt, precision_dt)
y_test_prob_knn=knn.predict_proba(X_test)[:,1]
precision_knn, recall_knn,_ =precision_recall_curve(Y_test, y_test_prob_knn)
auc_score_knn=metrics.auc(recall_knn, precision_knn)
y_test_prob_ann=MLPC.predict_proba(X_test)[:,1]
precision_ann, recall_ann,_ =precision_recall_curve(Y_test, y_test_prob_ann)
auc_score_ann=metrics.auc(recall_ann, precision_ann)
plt.figure(figsize=(5,5))
plt.plot(recall, precision, marker='',label="RFC="+str(auc_score_rf))
plt.plot(recall, precision, marker='',label="KNN="+str(auc_score_knn))
plt.plot(recall, precision, marker='',label="ANN="+str(auc_score_ann))
plt.plot(recall, precision, marker='',label="DTC="+str(auc_score_dt))
plt.plot(recall, precision, marker='',label="XGBC="+str(auc_score_xb))
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Recall-Precision Curve on RFC")
plt.legend()
plt.show()